In [ ]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import datetime
from zoneinfo import ZoneInfo

from constants import DATA_DIR

In [ ]:
path = os.path.join(DATA_DIR, "bus", "sf", "2025-03-09.parquet")
vehicle_df = pd.read_parquet(path, engine="fastparquet")

# Remove rows without valid stop_id
vehicle_df = vehicle_df[vehicle_df["vehicle.stop_id"].notnull()]
vehicle_df = vehicle_df.astype({"vehicle.stop_id": "int64"})

# Remove rows without valid trip_id and convert trip_id to int
vehicle_df = vehicle_df[vehicle_df["vehicle.trip.trip_id"].notnull()]
vehicle_df["vehicle.trip_id"] = vehicle_df.apply(lambda row: int(row["vehicle.trip.trip_id"].split("_")[0]), axis=1)

# Only keep necessary columns
vehicle_df = vehicle_df[["vehicle.trip_id", "vehicle.timestamp", "vehicle.stop_id", "vehicle.trip.route_id", "vehicle.trip.direction_id"]]

# Only keep the last record before a bus arrives 
vehicle_df.drop_duplicates(subset=["vehicle.trip_id", "vehicle.stop_id"], keep="last", inplace=True)

In [ ]:
path = os.path.join(DATA_DIR, "stop_times.txt")
stop_time_df = pd.read_csv(path)
stop_time_df = stop_time_df[["trip_id", "departure_time", "stop_id"]]

path = os.path.join(DATA_DIR, "stops.txt")
stop_df = pd.read_csv(path)
stop_df = stop_df[["stop_id", "stop_code"]]

# Add stop code info to stop times and only keep necessary columns
schedule_df = stop_time_df.join(stop_df.set_index("stop_id"), on="stop_id")[["trip_id", "departure_time", "stop_code"]]
del stop_time_df
del stop_df

# Drop all stops that appear twice in a trip to simplify comparisons
schedule_df.drop_duplicates(subset=["trip_id", "stop_code"], keep=False, inplace=True)

In [ ]:
delay_df = pd.merge(
    left=vehicle_df, 
    right=schedule_df, 
    left_on=["vehicle.trip_id", "vehicle.stop_id"], 
    right_on=["trip_id", "stop_code"],
    validate="1:1"
)[["trip_id", "stop_code", "vehicle.timestamp", "departure_time", "vehicle.trip.route_id", "vehicle.trip.direction_id"]]
del vehicle_df
del schedule_df
delay_df

In [ ]:
def get_delay(actual_time: datetime.datetime, scheduled_time: str) -> int:
    tz = ZoneInfo("America/Los_Angeles")
    t = scheduled_time.split(":")
    h = int(t[0]) % 24
    m = int(t[1])
    s = int(t[2])
    schedule_timestamp = actual_time.replace(hour=h, minute=m, second=s, tzinfo=tz)
    naive = schedule_timestamp.replace(tzinfo=None) - datetime.timedelta(1)
    schedule_timestamp_adjust = naive.replace(tzinfo=tz)
    dif = actual_time - schedule_timestamp
    dif_adjust = actual_time - schedule_timestamp_adjust
    if abs(dif_adjust) < abs(dif):
        dif = dif_adjust
        schedule_timestamp = schedule_timestamp_adjust
    return int(dif.total_seconds() / 60), schedule_timestamp.astimezone(datetime.UTC)


In [ ]:
delay_df[["delay", "schedule_time"]] = delay_df.apply(lambda row: get_delay(row["vehicle.timestamp"], row["departure_time"]), axis=1, result_type="expand")
delay_df = delay_df[abs(delay_df["delay"]) < 60]
delay_df = delay_df[["trip_id", "stop_code", "vehicle.timestamp", "schedule_time", "delay", "vehicle.trip.route_id", "vehicle.trip.direction_id"]]

In [ ]:
plt.hist(delay_df["delay"], bins=100)
plt.show()
delay_df